In [8]:
import random
import numpy as np

class Matriz:
    # construtor da classe, contém número de enfermeiros, número de turnos e a matriz (zerada)
    def __init__(self, nenfermeiros, nturnos, matriz = []):
        self.nenfermeiros = nenfermeiros # númeo de enfermeiros
        self.nturnos = nturnos # número de turnos
        self.enfermeiros = self.criaMatriz(nenfermeiros, nturnos) # própria matriz
        self.turnos = self.criaMatriz(nturnos, nenfermeiros) # representação diferente da matriz

# método para fazer uma matriz
    def criaMatriz(self, l, c):
        matriz = []
        for _ in range(l):
            linha = []
            for _ in range(c):
                linha.append(0)
            matriz.append(linha)
        return matriz

# retorna matriz como uma cadeia de 0s e 1s
    def printMatriz(self):
        string = ""
        for enf in range(self.nenfermeiros):
            for turno in range(self.nturnos):
                string += str(self.enfermeiros[enf][turno])
        return string

    def stringParaMatriz(self, string):
        it = 0
        for enf in range(self.nenfermeiros):
            for turno in range(self.nturnos):
                self.enfermeiros[enf][turno] = int(string[it])
                self.turnos[turno][enf] = int(string[it])
                it +=1

# função referente à restrição 1: min 1 enfermeiro e max 3 enfermeiros por turno
#  calcula o valor da penalidade e retorna
def r1(matriz):
    penalidade = 0 # armazena a penalidade

   # olha cada turno e checa se a quantidade de enfermeiros é menor que 1 ou maior que 3 para penalizar
    for turno in matriz.turnos:
        soma = sum(turno)
        if (soma < 1 or soma > 3):
            penalidade += -1

    return penalidade

# função referente à restrição 2: Cada enfermeiro deve ser alocado em 5 turnos por semana
# calcula o valor da penalidade e retorna
def r2(matriz):
    penalidade = 0 # armazena a penalidade

   # olha cada enfermeiros e checa se a quantidade de turnos que ele está é menor ou maior que 5 para penalizar
    for enfermeiro in matriz.enfermeiros:
        soma = sum(enfermeiro)
        if (soma < 5 or soma > 5):
            penalidade += -1

    return penalidade

# função referente à restrição 3: Nenhum enfermeiro pode trabalhar mais que 3 dias seguidos sem folga
# calcula o valor da penalidade e retorna
def r3(matriz):
    penalidade = 0 # armazena a penalidade
    diasTrabalhados = [] # auxiliar que verifica os dias trabalhados consecutivos
    nturno = 1 # auxiliar que verifica qual o turno

   # olha para cada turno de cada enfermeiro (em sequencia) e verifica se foi trabalhado em dias consecutivos
    for enfermeiro in matriz.enfermeiros:
        turnos = [] # auxiliar para verificar se o enfermeiro trabalhou naquele dia ou não
        for turno in enfermeiro:

           # guarda cada turno (a cada 3 turnos forma um dia)
            if (nturno <= 3):
                turnos.append(turno)

            # incrementa os dias
            if (nturno == 3) :
                nturno = 0 # garante que não passa de 3

                if (sum(turnos) > 0):
                    diasTrabalhados.append(1)
                else:
                    diasTrabalhados.append(0)
                    if (sum(diasTrabalhados) > 3):
                        penalidade += -1
                    diasTrabalhados.clear()
                turnos.clear()
            nturno += 1
        if (sum(diasTrabalhados) > 3):
                penalidade += -1

    return penalidade

# função referente à restrição 4
# calcula o valor da penalidade e retorna
def r4(matriz):
    penalidade = 0
    for enf in range(matriz.nenfermeiros):
        for turno in range(matriz.nturnos):
            enfermeiros = matriz.enfermeiros
            if enfermeiros[enf][turno] == "1":
                for turnoExtra in range(matriz.nturnos):
                    if turno != turnoExtra and enfermeiros[enf][turnoExtra]:
                        penalidade += -1
    return penalidade
# função que calcula o fitness de determinada matriz
def fitness(matriz, restricoes):
    fit = 0
    if (1 in restricoes):
        fit += r1(matriz)
    if (2 in restricoes):
        fit += r2(matriz)
    if (3 in restricoes):
            fit += r3(matriz)
    return fit

# Cruzamento (crossover)
# contem o parâmetro var que indica qual variação está sendo usada
def crossover(tampopulacao, populacao, melhores_indices, restricoes, var):
        # a partir dos melhores individuos, pegamos 2 e realizamos o crossover para criar 2 novos filhos
        novos_individuos = []
        for _ in range(tampopulacao - len(melhores_indices)): # conta sem os melhores individuos
            index1 = random.choice(melhores_indices)
            index2 = random.choice(melhores_indices)
            pai1 = populacao[index1].printMatriz()
            pai2 = populacao[index2].printMatriz()
            ponto_corte = random.randint(1, len(pai1)) # seleciona o ponto de corte
            # realiza o crossover (troca as partes de cada pai)
            filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
            filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
            if (var == 1):
                matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturnos)
                matriz.stringParaMatriz(filho1)
                novos_individuos.append(matriz)
            elif var == 2:
                matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturnos)
                matriz.stringParaMatriz(filho2)
                novos_individuos.append(matriz)
            elif var == 3:
                if (fitness(filho1, restricoes) > fitness(filho2, restricoes)):
                    matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturno)
                    matriz.stringParaMatriz(filho1)
                    novos_individuos.append(matriz)
                else:
                    matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturno)
                    matriz.stringParaMatriz(filho2)
                    novos_individuos.append(matriz)
        return novos_individuos

# Mutação
def mutacaoF(mutacao, novos_individuos):
   # realiza a mutação alterando um caractere em uma cadeia (se é 0, muda para 1 e vice-versa)
    for novo in novos_individuos:
        # aplica a mutação de acordo com a taxa de mutação
        if random.random() < mutacao:
            novoStr = novo.printMatriz()
            index = random.randint(0, len(novoStr) - 1)
            string = ""
            if (novoStr[index] == "0"):
                if (index == 0):
                    string = "1" + novoStr[index+1:]
                elif index == (len(novoStr) - 1):
                    string = novoStr[:index]+ "1"
                else:
                    string = novoStr[:index] + "1" + novoStr[index+1:]
            else:
                if (index == 0):
                    string = "0" + novoStr[index+1:]
                elif index == (len(novoStr) - 1):
                    string = novoStr[:index]+ "0"
                else:
                    string = novoStr[:index] + "0" + novoStr[index+1:]
            novoStr = string
            novo.stringParaMatriz(novoStr)
    return novos_individuos

def algGenetico(nenfermeiros, nturno,restricoes, tampopulacao, interacoes, elitismo, mutacao):
    # cria strings aleatorias que representam um cromossomo (uma possivel configuração)
    def string_aleatoria(tamanho):
        return ''.join(random.choice('01') for _ in range(tamanho))
    # armazena a população (inicialmente vazia)
    populacao = []
    for _ in range(tampopulacao):
        matriz = Matriz(nenfermeiros=nenfermeiros, nturnos=nturno)
        matriz.stringParaMatriz(string_aleatoria(nenfermeiros*nturno)) # preenche a matriz
        populacao.append(matriz)

    for _ in range(interacoes):
        # armazena o fitness de cada elemento da população
        fitnessPopulacao = []
        fitnesspositivo = []
        for matriz in populacao:
            value = fitness(matriz,restricoes)
            fitnessPopulacao.append(value)
            fitnesspositivo.append(value *(-1))
        novos_individuos= populacao
        melhores_individuos = []
        if (elitismo > 0):
          # escolha dos melhores indivíduos (elitismo)
          melhores_indices = np.argsort(fitnesspositivo)[:int(elitismo * tampopulacao)] # pega os indices dos melhores fitness

          for indice in melhores_indices:
              melhores_individuos.append(populacao[indice]) # pega os elementos correspondentes aos indices

          # realiza o crossover
          novos_individuos = crossover(tampopulacao=tampopulacao, melhores_indices=melhores_indices, restricoes=restricoes, var=1, populacao=populacao)

        # realiza a mutação
        novos_individuos = mutacaoF(mutacao=mutacao, novos_individuos=novos_individuos)
        for individuo in novos_individuos:
            melhores_individuos.append(individuo)
        populacao = melhores_individuos

    # Encontrando a melhor solução
    melhor_indice = np.argmax(fitnessPopulacao)
    melhor_solucao = populacao[melhor_indice]
    return melhor_solucao.printMatriz(), fitness(melhor_solucao, restricoes)

def main():
    #leitura dos dados
    nenfermeiros = 10

    nturnos = 21

    restricoes = [1,2,3,4]

    tampopulacao = 100

    interacoes = 1000

    mutacao = 0.1
    for el in [0,0.1,0.25,0.5,0.75]:
      print("ELITISMO:", el)
      for _ in range(10):
        cadeia, fit = algGenetico(nenfermeiros=nenfermeiros, nturno=nturnos, restricoes=restricoes, tampopulacao=tampopulacao, interacoes=interacoes, elitismo=el, mutacao=mutacao)
        print(cadeia)
        print(fit)

if __name__ == "__main__":
    main()



ELITISMO: 0
000010101000100000100001011111000111010110100010101111100001111001010101010101100111010100111011101000111101010100011001110011011110001100111010111101000100010001001000001001101111001110111011001111010011000000
-31
011110000000110010000110100000110011110000100010100110010100011000000101110011011010111001011100010101100010010001010001011101011001100100000101010000101000111100111010100100100110000000000011100110001000000000
-30
010001000001010000110000000101001000110100100101010100000100001111101101010101000010010000111101000100100001101001100100101011111001011010101111101111010000011100000110100011000011000111110111110011001000000110
-29
000101000011000011111010011110011011011000000110010101010000111000100110100000000010101110001011010100010011010000001010101101100011100001110010000110101000111000111011000000001110000011100100010100000001111011
-31
00110000000110000011001110101011101101010111011101000011100001010100100110001000001000000110011100101011110110110100100001001110

In [ ]:
import random
import numpy as np

class Matriz:
    # construtor da classe, contém número de enfermeiros, número de turnos e a matriz (zerada)
    def __init__(self, nenfermeiros, nturnos, matriz = []):
        self.nenfermeiros = nenfermeiros # númeo de enfermeiros
        self.nturnos = nturnos # número de turnos
        self.enfermeiros = self.criaMatriz(nenfermeiros, nturnos) # própria matriz
        self.turnos = self.criaMatriz(nturnos, nenfermeiros) # representação diferente da matriz

# método para fazer uma matriz
    def criaMatriz(self, l, c):
        matriz = []
        for _ in range(l):
            linha = []
            for _ in range(c):
                linha.append(0)
            matriz.append(linha)
        return matriz

# retorna matriz como uma cadeia de 0s e 1s
    def printMatriz(self):
        string = ""
        for enf in range(self.nenfermeiros):
            for turno in range(self.nturnos):
                string += str(self.enfermeiros[enf][turno])
        return string

    def stringParaMatriz(self, string):
        it = 0
        for enf in range(self.nenfermeiros):
            for turno in range(self.nturnos):
                self.enfermeiros[enf][turno] = int(string[it])
                self.turnos[turno][enf] = int(string[it])
                it +=1

# função referente à restrição 1: min 1 enfermeiro e max 3 enfermeiros por turno
#  calcula o valor da penalidade e retorna
def r1(matriz):
    penalidade = 0 # armazena a penalidade

   # olha cada turno e checa se a quantidade de enfermeiros é menor que 1 ou maior que 3 para penalizar
    for turno in matriz.turnos:
        soma = sum(turno)
        if (soma < 1 or soma > 3):
            penalidade += -1

    return penalidade

# função referente à restrição 2: Cada enfermeiro deve ser alocado em 5 turnos por semana
# calcula o valor da penalidade e retorna
def r2(matriz):
    penalidade = 0 # armazena a penalidade

   # olha cada enfermeiros e checa se a quantidade de turnos que ele está é menor ou maior que 5 para penalizar
    for enfermeiro in matriz.enfermeiros:
        soma = sum(enfermeiro)
        if (soma < 5 or soma > 5):
            penalidade += -1

    return penalidade

# função referente à restrição 3: Nenhum enfermeiro pode trabalhar mais que 3 dias seguidos sem folga
# calcula o valor da penalidade e retorna
def r3(matriz):
    penalidade = 0 # armazena a penalidade
    diasTrabalhados = [] # auxiliar que verifica os dias trabalhados consecutivos
    nturno = 1 # auxiliar que verifica qual o turno

   # olha para cada turno de cada enfermeiro (em sequencia) e verifica se foi trabalhado em dias consecutivos
    for enfermeiro in matriz.enfermeiros:
        turnos = [] # auxiliar para verificar se o enfermeiro trabalhou naquele dia ou não
        for turno in enfermeiro:

           # guarda cada turno (a cada 3 turnos forma um dia)
            if (nturno <= 3):
                turnos.append(turno)

            # incrementa os dias
            if (nturno == 3) :
                nturno = 0 # garante que não passa de 3

                if (sum(turnos) > 0):
                    diasTrabalhados.append(1)
                else:
                    diasTrabalhados.append(0)
                    if (sum(diasTrabalhados) > 3):
                        penalidade += -1
                    diasTrabalhados.clear()
                turnos.clear()
            nturno += 1
        if (sum(diasTrabalhados) > 3):
                penalidade += -1

    return penalidade

# função referente à restrição 4
# calcula o valor da penalidade e retorna
def r4(matriz):
    penalidade = 0
    for enf in range(matriz.nenfermeiros):
        for turno in range(matriz.nturnos):
            enfermeiros = matriz.enfermeiros
            if enfermeiros[enf][turno] == "1":
                for turnoExtra in range(matriz.nturnos):
                    if turno != turnoExtra and enfermeiros[enf][turnoExtra]:
                        penalidade += -1
    return penalidade
# função que calcula o fitness de determinada matriz
def fitness(matriz, restricoes):
    fit = 0
    if (1 in restricoes):
        fit += r1(matriz)
    if (2 in restricoes):
        fit += r2(matriz)
    if (3 in restricoes):
            fit += r3(matriz)
    return fit

# Cruzamento (crossover)
# contem o parâmetro var que indica qual variação está sendo usada
def crossover(tampopulacao, populacao, melhores_indices, restricoes, var):
        # a partir dos melhores individuos, pegamos 2 e realizamos o crossover para criar 2 novos filhos
        novos_individuos = []
        for _ in range(tampopulacao - len(melhores_indices)): # conta sem os melhores individuos
            index1 = random.choice(melhores_indices)
            index2 = random.choice(melhores_indices)
            pai1 = populacao[index1].printMatriz()
            pai2 = populacao[index2].printMatriz()
            ponto_corte = random.randint(1, len(pai1)) # seleciona o ponto de corte
            # realiza o crossover (troca as partes de cada pai)
            filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
            filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
            if (var == 1):
                matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturnos)
                matriz.stringParaMatriz(filho1)
                novos_individuos.append(matriz)
            elif var == 2:
                matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturnos)
                matriz.stringParaMatriz(filho2)
                novos_individuos.append(matriz)
            elif var == 3:
                if (fitness(filho1, restricoes) > fitness(filho2, restricoes)):
                    matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturno)
                    matriz.stringParaMatriz(filho1)
                    novos_individuos.append(matriz)
                else:
                    matriz = Matriz(nenfermeiros=populacao[index1].nenfermeiros, nturnos=populacao[index1].nturno)
                    matriz.stringParaMatriz(filho2)
                    novos_individuos.append(matriz)
        return novos_individuos

# Mutação
def mutacaoF(mutacao, novos_individuos):
   # realiza a mutação alterando um caractere em uma cadeia (se é 0, muda para 1 e vice-versa)
    for novo in novos_individuos:
        # aplica a mutação de acordo com a taxa de mutação
        if random.random() < mutacao:
            novoStr = novo.printMatriz()
            index = random.randint(0, len(novoStr) - 1)
            string = ""
            if (novoStr[index] == "0"):
                if (index == 0):
                    string = "1" + novoStr[index+1:]
                elif index == (len(novoStr) - 1):
                    string = novoStr[:index]+ "1"
                else:
                    string = novoStr[:index] + "1" + novoStr[index+1:]
            else:
                if (index == 0):
                    string = "0" + novoStr[index+1:]
                elif index == (len(novoStr) - 1):
                    string = novoStr[:index]+ "0"
                else:
                    string = novoStr[:index] + "0" + novoStr[index+1:]
            novoStr = string
            novo.stringParaMatriz(novoStr)
    return novos_individuos

def algGenetico(nenfermeiros, nturno,restricoes, tampopulacao, interacoes, elitismo, mutacao):
    # cria strings aleatorias que representam um cromossomo (uma possivel configuração)
    def string_aleatoria(tamanho):
        return ''.join(random.choice('01') for _ in range(tamanho))
    # armazena a população (inicialmente vazia)
    populacao = []
    for _ in range(tampopulacao):
        matriz = Matriz(nenfermeiros=nenfermeiros, nturnos=nturno)
        matriz.stringParaMatriz(string_aleatoria(nenfermeiros*nturno)) # preenche a matriz
        populacao.append(matriz)

    for _ in range(interacoes):
        # armazena o fitness de cada elemento da população
        fitnessPopulacao = []
        fitnesspositivo = []
        for matriz in populacao:
            value = fitness(matriz,restricoes)
            fitnessPopulacao.append(value)
            fitnesspositivo.append(value *(-1))
        novos_individuos= populacao
        melhores_individuos = []
        if (elitismo > 0):
          # escolha dos melhores indivíduos (elitismo)
          melhores_indices = np.argsort(fitnesspositivo)[:int(elitismo * tampopulacao)] # pega os indices dos melhores fitness

          for indice in melhores_indices:
              melhores_individuos.append(populacao[indice]) # pega os elementos correspondentes aos indices

          # realiza o crossover
          novos_individuos = crossover(tampopulacao=tampopulacao, melhores_indices=melhores_indices, restricoes=restricoes, var=1, populacao=populacao)

        # realiza a mutação
        novos_individuos = mutacaoF(mutacao=mutacao, novos_individuos=novos_individuos)
        for individuo in novos_individuos:
            melhores_individuos.append(individuo)
        populacao = melhores_individuos

    # Encontrando a melhor solução
    melhor_indice = np.argmax(fitnessPopulacao)
    melhor_solucao = populacao[melhor_indice]
    return melhor_solucao.printMatriz(), fitness(melhor_solucao, restricoes)

def main():
    #leitura dos dados
    nenfermeiros = 10

    nturnos = 21

    restricoes = [1,2,3,4]

    elitismo = 0.1

    interacoes = 1000

    mutacao = 0.1
    for tam in [10, 25, 50, 100, 500, 1000]:
      print("TAMANHO DA POPULAÇÃO:", tam)
      for _ in range(10):
        cadeia, fit = algGenetico(nenfermeiros=nenfermeiros, nturno=nturnos, restricoes=restricoes, tampopulacao=tam, interacoes=interacoes, elitismo=elitismo, mutacao=mutacao)
        print(cadeia)
        print(fit)

if __name__ == "__main__":
    main()



TAMANHO DA POPULAÇÃO: 10
010100111000011011000000001111101000111110110101011000000001000110010001011110110100010101011011100000010111110000001001000000011001000011000110101010000010000001011101010101111001110011000000010000001000010110
-22
011000101010000000000101101111000001011101000010000000001000111000000110110001000000000001100110000001011000111001111000101001010001011110101101001010010011011111000010100100111111010010001001001100010100101100
-20
011000100100100000000100000100000010110000101000101110000100100110101110110001110110101001100100000100001100100001110100101111100010100100001000000111000100001000111110000111001111000100100001000000110001000100
-17
101100110000000110000101100010000100011011001110110100010011000001101110000001011000111111110001000010000000000000000011101010000011101000011011000010101100000001011111110011000000011011100100010110110110101101
-24
1000110001011110000010001001111011010110111111001111101110110010100100111100111100001110010110001110010001010011110